## 5日目

### モナドがいっぱい
* モナドはある願いを叶えるための、アプリカティブ値の自然な拡張
* 「普通の値 a をとって、文脈付きの値を返す関数に、文脈付きの値 m a を渡したい」

In [1]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

* モナドの型コンストラクト
    - Applicative と Bind を拡張している

```scala
trait Monad[F[_]] extends Applicative[F] with Bind[F] { self =>
  ////
}
```

* Bindのコンストラクト

```scala
trait Bind[F[_]] extends Apply[F] { self =>
    def bind[A, B](fa: F[A])(f: A => F[B]): F[B]
}
```

In [2]:
3.some flatMap { x => (x + 1).some }

res1: Option[Int] = Some(4)

In [3]:
(none: Option[Int]) flatMap { x => (x + 1).some }

res2: Option[Int] = None

* Bindは、Optionalとかで包まれている場合でも、その中身を操作できるってこと?

In [4]:
Monad[Option].point("WHAT")

res3: Option[String] = Some("WHAT")

In [5]:
9.some flatMap { x => Monad[Option].point(x * 10) }

res4: Option[Int] = Some(90)

In [6]:
(none: Option[Int]) flatMap {x => Monad[Option].point(x * 10) }

res5: Option[Int] = None

### 綱渡り
* H本の　Pole　の例題とやらを実装する

In [7]:
type Birds = Int

defined type Birds

In [8]:
case class Pole(left: Birds, right: Birds)

defined class Pole

In [9]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Pole = copy(left = left + n)
    def landRight(n: Birds): Pole = copy(right = right + n)
}

defined class Pole

In [10]:
Pole(0, 0).landLeft(2)

res9: Pole = Pole(2, 0)

In [11]:
Pole(1, 2).landRight(1)

res10: Pole = Pole(1, 3)

In [12]:
Pole(1, 2).landRight(-1)

res11: Pole = Pole(1, 1)

In [13]:
Pole(0, 0).landLeft(1).landRight(1).landLeft(2)

res12: Pole = Pole(3, 1)

In [14]:
Pole(0, 0).landLeft(1).landRight(4).landLeft(-1).landRight(-2)

res13: Pole = Pole(0, 2)

In [16]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Option[Pole] =
        if (math.abs((left + n) - right) < 4) copy(left = left + n).some
        else none
    def landRight(n: Birds): Option[Pole] =
        if (math.abs(left - (right + n)) < 4) copy(right = right + n).some
        else none
}

defined class Pole

In [17]:
Pole(0, 0).landLeft(2)
Pole(0, 3).landLeft(10)

res16_0: Option[Pole] = Some(Pole(2, 0))
res16_1: Option[Pole] = None

In [18]:
Pole(0, 0).landRight(1) flatMap {_.landLeft(2)}

res17: Option[Pole] = Some(Pole(2, 1))

In [19]:
(none: Option[Pole]) flatMap {_.landLeft(2)}

res18: Option[Pole] = None

In [20]:
Monad[Option].point(Pole(0, 0)) flatMap {_.landRight(2)} flatMap {_.landLeft(2)}

res19: Option[Pole] = Some(Pole(2, 2))

In [22]:
Monad[Option].point(Pole(0, 0)) >>= {_.landRight(2)} >>= {_.landLeft(2)} >>=  {_.landRight(2)}

res21: Option[Pole] = Some(Pole(2, 4))

In [23]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >>= {_.landRight(4)} >>=  {_.landLeft(-1)} >>= {_.landRight(-2)}

res22: Option[Pole] = None

In [27]:
Pole(0, 0).some >>= {_.landLeft(1)} >>= {_.landRight(4)} >>=  {_.landLeft(-1)} >>= {_.landRight(-2)}

res26: Option[Pole] = None

* Option で包んであっても Functor 使えば別に大丈夫だった
* `Monad[Option].point(Pole(0, 0))` は `Pole(0, 0).some` と同じ結果なら別にメリット感じなくない?

In [31]:
Monad[Option].point(Pole(0, 0))
Pole(0, 0).some

res30_0: Option[Pole] = Some(Pole(0, 0))
res30_1: Option[Pole] = Some(Pole(0, 0))

### ロープの上のバナナ
* 鳥の数に関係なく、いきなりピエールを滑らせて落っことす

In [32]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Option[Pole] =
        if (math.abs((left + n) - right) < 4) copy(left = left + n).some
        else none
    def landRight(n: Birds): Option[Pole] =
        if (math.abs(left - (right + n)) < 4) copy(right = right + n).some
        else none
    def banana: Option[Pole] = none
}

defined class Pole

In [33]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >>= {_.banana} >>=  {_.landRight(1)}

res32: Option[Pole] = None

* 入力に関係なく規定のモナド値を返す関数だったら、自作せずにも `>>` 関数を使える

In [34]:
(none: Option[Int]) >> 3.some

res33: Option[Int] = None

In [35]:
3.some >> 4.some

res34: Option[Int] = Some(4)

In [36]:
3.some >> (none: Option[Int])

res35: Option[Int] = None

In [37]:
3.some |+| 4.some

res36: Option[Int] = Some(7)

In [38]:
4.some >> 3.some

res37: Option[Int] = Some(3)

In [38]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >> (none: Option[Pole]) >>= {_.landRight(1)}

cmd38.sc:1: missing parameter type for expanded function ((x$1) => x$1.landLeft(1))
val res38 = Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >> (none: Option[Pole]) >>= {_.landRight(1)}
                                                 ^

: 

In [39]:
Monad[Option].point(Pole(0, 0)).>>=({_.landLeft(1)}).>>(none: Option[Pole]).>>=({_.landRight(1)})

res38: Option[Pole] = None

In [40]:
(Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)}) >> (none: Option[Pole]) >>= {_.landRight(1)}

res39: Option[Pole] = None

### for 構文
* Haskell にとってモナドはとても便利なので、モナド専用構文まで用意されている
* do 記法

In [47]:
3.some >>= { x => "!".some >>= { y => (x.shows + y).some } }

res46: Option[String] = Some("3!")

In [43]:
1.some >>= {x => (none: Option[String]) >>= { y => (x.shows + y).some } }

res42: Option[String] = None

In [44]:
(none: Option[Int]) >>= {x => "!".some >>= { y => (x.shows + y).some } }

res43: Option[String] = None

In [45]:
3.some >>= {x => "!".some >>= { y => (none: Option[String]) } }

res44: Option[String] = None

In [48]:
for {
    x <- 3.some
    y <- "!".some
} yield (x.shows + y)

res47: Option[String] = Some("3!")

* do 式は let 行を除いて全てモナド値で構成される
* Scala の for 構文もおそらくそう

In [49]:
def routine: Option[Pole] =
  for {
      start <- Monad[Option].point(Pole(0, 0))
      first <- start.landLeft(2)
      second <- first.landRight(2)
      third <- second.landLeft(1)
  } yield third

defined function routine

In [50]:
routine

res49: Option[Pole] = Some(Pole(3, 2))

In [51]:
def routine: Option[Pole] =
  for {
      start <- Monad[Option].point(Pole(0, 0))
      first <- start.landLeft(2)
      _ <- (none: Option[Pole])
      second <- first.landRight(2)
      third <- second.landLeft(1)
  } yield third

defined function routine

In [52]:
routine

res51: Option[Pole] = None

### パターンマッチングと失敗

In [53]:
def justH: Option[Char] =
  for {
      (x :: xs) <- "hello".toList.some
  } yield x

defined function justH

In [54]:
justH

res53: Option[Char] = Some('h')

In [55]:
def wopwop: Option[Char] =
  for {
      (x :: xs) <- "".toList.some
  } yield x

defined function wopwop

In [56]:
wopwop

res55: Option[Char] = None

### List モナド

In [57]:
^(List(1, 2, 3), List(10, 100, 100)) {_ * _}

res56: List[Int] = List(10, 100, 100, 20, 200, 200, 30, 300, 300)

In [58]:
List(3, 4, 5) >>= {x => List(x, -x)}

res57: List[Int] = List(3, -3, 4, -4, 5, -5)

In [59]:
for {
    n <- List(1, 2)
    ch <- List('a', 'b')
} yield (n, ch)

res58: List[(Int, Char)] = List((1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'))

### MonadPlus と guard 関数
* Scala の for 構文はフィルタリングができる

In [60]:
for {
    x <- 1 |-> 50 if x.shows contains '7'
} yield x

res59: List[Int] = List(7, 17, 27, 37, 47)

* MonadPlus は、モノイドの性質を併せ持つモナドを表す型クラス

* MonadPlus の型クラスのコントラクト

```scala
trait MonadPlus[F[_]] extends Monad[F] with ApplicativePlus[F] { self =>
    ...
}
```

### Plus, PlusEmpty と ApplicativePlus
* これは ApplicativePlus を継承している

```scala
trait ApplicativePlus[F[_]] extends Applicative[F] with PlusEmpty[F] { self =>
  ...
}
```

* そしてそれは、 PlusEmpty を継承している

```scala
trait PlusEmpty[F[_]] extends Plus[F] { self =>
  ////
  def empty[A]: F[A]
}
```

* Plus は２つのコンテナを連結する `<+>` 演算子を導入する

In [61]:
List(1, 2, 3) <+> List(4, 5, 6)

res60: List[Int] = List(1, 2, 3, 4, 5, 6)

In [62]:
(1 |-> 50) filter { x => x.shows contains '7' }

res61: List[Int] = List(7, 17, 27, 37, 47)

### 騎士の旅

ここで、非決定性計算を使って解くのにうってつけの問題をご紹介しましょう。チェス盤の上にナイトの駒が1つだけ乗っています。
ナイトを３回動かして特定のマスまで移動させられるか、というのが問題です。

In [63]:
case class KnightPos(c: Int, r: Int)

defined class KnightPos

In [64]:
case class KnightPos(c: Int, r: Int) {
    def move: List[KnightPos] =
      for {
          KnightPos(c2, r2) <- List(KnightPos(c + 2, r - 1), KnightPos(c + 2, r + 1),
                                    KnightPos(c - 2, r - 1), KnightPos(c - 2, r + 1),
                                    KnightPos(c + 1, r - 2), KnightPos(c + 1, r + 2),
                                    KnightPos(c - 1, r - 2), KnightPos(c - 1, r + 2)) if (
          ((1 |-> 8) contains c2) && ((1 |-> 8) contains r2))
      } yield KnightPos(c2, r2)
}

defined class KnightPos

In [65]:
KnightPos(6, 2).move

res64: List[KnightPos] = List(
  KnightPos(8, 1),
  KnightPos(8, 3),
  KnightPos(4, 1),
  KnightPos(4, 3),
  KnightPos(7, 4),
  KnightPos(5, 4)
)

In [66]:
KnightPos(8, 1).move

res65: List[KnightPos] = List(KnightPos(6, 2), KnightPos(7, 3))

In [67]:
case class KnightPos(c: Int, r: Int) {
    def move: List[KnightPos] =
      for {
          KnightPos(c2, r2) <- List(KnightPos(c + 2, r - 1), KnightPos(c + 2, r + 1),
                                   KnightPos(c - 2, r - 1), KnightPos(c - 2, r + 1),
                                   KnightPos(c + 1, r - 2), KnightPos(c + 1, r + 2),
                                   KnightPos(c - 1, r - 2), KnightPos(c - 1, r + 2)) if (
          ((1 |-> 8) element c2) && ((1 |-> 8) contains r2))
      } yield KnightPos(c2, r2)
    def in3: List[KnightPos] =
      for {
          first <- move
          second <- first.move
          third <- second.move
      } yield third
    def canReachIn3(end: KnightPos): Boolean = in3 contains end
}

defined class KnightPos

In [68]:
KnightPos(6, 2) canReachIn3 KnightPos(6, 1)

res67: Boolean = true

In [69]:
KnightPos(6, 2) canReachIn3 KnightPos(7, 3)

res68: Boolean = false

### Monad 則
#### 左単位元
第一のモナド則が言っているのは、 return を使って値をデフォルトの文脈に入れたものを `>>=` を使って関数に食わせた結果は、単にその値にその関数を適用した結果と等しくなりなさい、ということです。

In [75]:
(Monad[Option].point(3) >>= { x => (x + 100000).some }) assert_=== (3.some flatMap { x => (x + 100000).some })

#### 右単位元
モナドの第二法則は、 `>>=` を使ってモナド値を return に食わせた結果は、元のモナド値と不変であると言っています。

In [71]:
("move on up".some flatMap {Monad[Option].point(_)}) assert_=== "move on up".some

#### 結合律
最後のモナド則は、 `>>=` を使ったモナド関数適用の連鎖があるときに、どの順序で評価しても結果は同じであるべき、というものです。

In [76]:
Monad[Option].point(Pole(0, 0)) >>= {_.landRight(2)} >>= {_.landLeft(2)} >>= {_.landRight(2)}

res75: Option[Pole] = Some(Pole(2, 4))

In [77]:
Monad[Option].point(Pole(0, 0)) >>= { x =>
  x.landRight(2) >>= { y =>
  y.landLeft(2) >>= { z =>
  z.landRight(2)
}}}

res76: Option[Pole] = Some(Pole(2, 4))

Scalaz 7 はモナド則を以下のように表現する

```scala
trait MonadLaw extends ApplicativeLaw {
    def rightIdentity[A](a: F[A])(implicit FA: Equal[F[A]]): Boolean = FA.equal(bind(a)(point(_: A)), a)
    def leftIdentity[A, B](a: A, f: A => F[B])(implicit FB: Equal[F[B]]): Boolean = FB.equal(bind(point(a))(f), F(a))
    def associativeBind[A, B, C](fa: F[A], f: A => F[B], g: B => F[C])(implicit FC: Equal[F[C]]): Boolean =
      FC.equal(bind(bind(fa)(f))(g), bind(fa)((a: A) => bind(f(a))(g)))
}
```